### Import the pacakges

In [1]:
from fastai.vision.all import * 
import tifffile as tiff

### Define the source directory for the raw data

In [2]:
# Replace with your raw data folder 
src = 'C:/Users/susa03/Documents/Post_project/Dataset/synthetic_data/raw_data'

### Define the destination directory and then create four subfolders for BSE, SE, hyperstack_images and masks to store

In [6]:
dst = 'C:/Users/susa03/Documents/Post_project/Dataset/synthetic_data/sorted_data/'
bse = 'bse'   
se = 'se'
images = 'hyperstack'
masks = 'masks'     ### Store ground truth image
os.makedirs(os.path.join(dst,bse), exist_ok=True)  # create destination folder if not exists
os.makedirs(os.path.join(dst,se), exist_ok=True)  # create destination folder if not exists
os.makedirs(os.path.join(dst,images), exist_ok=True)  # create destination folder if not exists
os.makedirs(os.path.join(dst,masks), exist_ok=True)  # create destination folder if not exists

# Get the full path of the created directory
bse_path = os.path.join(dst, bse)
print("BSE path:", bse_path)

se_path = os.path.join(dst, se)
print("SE path:", se_path)

image_path = os.path.join(dst, images)
print("Hyperstacked_Image path:", image_path)

masks_path = os.path.join(dst, masks)
print("Mask path:", masks_path)


BSE path: C:/Users/susa03/Documents/Post_project/Dataset/synthetic_data/sorted_data/bse
SE path: C:/Users/susa03/Documents/Post_project/Dataset/synthetic_data/sorted_data/se
Hyperstacked_Image path: C:/Users/susa03/Documents/Post_project/Dataset/synthetic_data/sorted_data/hyperstack
Mask path: C:/Users/susa03/Documents/Post_project/Dataset/synthetic_data/sorted_data/masks


### Assess the image files and define the image height and width accordingly

In [4]:
image_files = get_image_files(src)
file_count = len(image_files)
digit_count = len(str(file_count))
image_height = 850
image_width = 850

### Define the fuctions to assess each images category wise and then store as bse, se, hyperstack for the 7 channels in the hyperstack subfolder and the respective mask in the mask folder

In [5]:
def return_cat(filename:str,category:str):
    value = str(category) in str(filename)
    return value
   
def make_category_list(filenames):
 
    # Iterate over the images in groups of 8
    group_size = 8
    count = 0
    for i in range(0, len(image_files), group_size):
        image_group = image_files[i:i+group_size]
        bse_image= np.zeros((image_height, image_width)) 
        se_image= np.zeros((image_height, image_width)) 
        hyperstack_image= np.zeros((image_height, image_width,7)) 
        current_mask = np.zeros((image_height, image_width)) 
    
        # Process the current group of images
        for file in image_group:
            if return_cat(file,category='_bse'):
                bse_image_address = os.path.join(src,file)
                bse_image = tiff.imread(bse_image_address)
            if return_cat(file,category='_se'):
                se_image_address = os.path.join(src,file)
                se_image = tiff.imread(se_image_address) 
            if return_cat(file,category='entry_hf_0'):
                hyperstack_image_address = os.path.join(src,file)
                bse_en0 = tiff.imread(hyperstack_image_address)
                hyperstack_image[:,:,0] = bse_en0

            elif return_cat(file,category='entry_hf_1'):
                hyperstack_image_address = os.path.join(src,file)
                bse_en1 = tiff.imread(hyperstack_image_address)
                hyperstack_image[:,:,1] = bse_en1

            elif return_cat(file,category='exit_hf_0'):
                hyperstack_image_address = os.path.join(src,file)
                bse_ex0 = tiff.imread(hyperstack_image_address)
                hyperstack_image[:,:,2] = bse_ex0

            elif return_cat(file,category='exit_hf_1'):
                hyperstack_image_address = os.path.join(src,file)
                bse_ex1 = tiff.imread(hyperstack_image_address)
                hyperstack_image[:,:,3] = bse_ex1

            elif return_cat(file,category='normal'):
                hyperstack_image_address = os.path.join(src,file)
                normal = tiff.imread(hyperstack_image_address)
                hyperstack_image[:,:,4:7] = normal
             
            elif return_cat(file,category='gt'):
                current_mask_address = os.path.join(src,file)
                current_mask = tiff.imread(current_mask_address)
                count= count+1
            else: pass

        bse_image = bse_image.astype(np.uint8)
        bse_image_name = bse_path + '\\' + f"{count:0{digit_count}d}.tif"         
        tiff.imwrite(bse_image_name, bse_image)

        se_image = se_image.astype(np.uint8)
        se_image_name = se_path + '\\' + f"{count:0{digit_count}d}.tif"         
        tiff.imwrite(se_image_name, se_image)

        hyperstack_image = hyperstack_image.astype(np.uint8)
        hyperstack_image_name = image_path + '\\' + f"{count:0{digit_count}d}.tif"         
        tiff.imwrite(hyperstack_image_name, np.transpose(hyperstack_image,(2,0,1)))

        current_mask = current_mask.astype(np.uint8)
        current_mask_name = masks_path + '\\' + f"{count:0{digit_count}d}.tif"          
        tiff.imwrite(current_mask_name,current_mask)

make_category_list(image_files)